In [20]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [21]:
CEREBRAS_API_KEY=os.getenv('CEREBRAS_API_KEY')
TAVILY_API_KEY=os.getenv('TAVILY_API_KEY')
LANGSMITH_API_KEY = os.getenv('LANGSMITH_API_KEY')
LANGSMITH_TRACING=os.getenv('LANGSMITH_TRACING')
LANGSMITH_ENDPOINT=os.getenv('LANGSMITH_ENDPOINT')
LANGSMITH_API_KEY=os.getenv('LANGSMITH_API_KEY')
LANGSMITH_PROJECT=os.getenv('LANGSMITH_PROJECT')

In [22]:
os.environ['CEREBRAS_API_KEY'] = CEREBRAS_API_KEY
os.environ['TAVILY_API_KEY'] = TAVILY_API_KEY
os.environ['LANGSMITH_API_KEY'] = LANGSMITH_API_KEY
os.environ['LANGSMITH_TRACING'] = "true"
os.environ['LANGSMITH_ENDPOINT'] = "https://api.smith.langchain.com"
os.environ['LANGSMITH_PROJECT'] = LANGSMITH_PROJECT

In [11]:
from langchain_cerebras import ChatCerebras

llm = ChatCerebras(
    model="llama-3.3-70b",
)

In [25]:
while True:
    question = input("enter a prompt, to quit press q: ")
    if question != "quit":
        print(llm.invoke(question).content)
    else:
        print("goodbye!")
        break

enter a prompt, to quit press q:  hello, respond in 5 words. How can i surpass the limit of being a human


Merge with artificial intelligent systems.


enter a prompt, to quit press q:  quit


goodbye!


In [26]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.messages import AIMessage

In [27]:
store = {}

In [30]:
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

In [29]:
config = {"configurable": {"session_id": "firstchat"}}

In [32]:
model_with_memory = RunnableWithMessageHistory(llm, get_session_history)

In [33]:
model_with_memory.invoke(("Hi, i am Ochi Ojie"), config=config).content

"Hello Ochi Ojie, it's nice to meet you. Is there something I can help you with or would you like to chat?"

In [34]:
model_with_memory.invoke(("what is my name?"), config=config).content

'Your name is Ochi Ojie.'

In [35]:
store

{'firstchat': InMemoryChatMessageHistory(messages=[HumanMessage(content='Hi, i am Ochi Ojie', additional_kwargs={}, response_metadata={}), AIMessage(content="Hello Ochi Ojie, it's nice to meet you. Is there something I can help you with or would you like to chat?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 43, 'total_tokens': 72, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'llama-3.3-70b', 'system_fingerprint': 'fp_a4e4dbcbd01d96252206', 'id': 'chatcmpl-9f5ea9fb-677f-4c81-a21a-6a082ef05611', 'finish_reason': 'stop', 'logprobs': None}, id='run-3edd5e83-a9b7-425d-a5f9-4852f779c57a-0', usage_metadata={'input_tokens': 43, 'output_tokens': 29, 'total_tokens': 72, 'input_token_details': {}, 'output_token_details': {}}), HumanMessage(content='what is my name?', additional_kwargs={}, response_metadata={}), AIMessage(content='Your name is Ochi Ojie.', additional_kwargs={'refusal': None},

# RAG with LCEL

In [40]:
from langchain_community.document_loaders import TextLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain import PromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough , RunnableLambda
from langchain_core.output_parsers import StrOutputParser

In [41]:
## Reading the txt files from source directory

loader = DirectoryLoader('../data', glob="./".txt", loader_cls = TextLoader)
docs = loader.load()